In [1]:
import globus_automate_client
import mdf_toolbox

In [16]:
native_app_id = "417301b1-5101-456a-8a27-423e71a2ae26"  # Premade native app ID (from CFDE)
flows_client = globus_automate_client.create_flows_client(native_app_id)

In [21]:
transfer_loop_flow_def = {
    "title": "Transfer Action Loop Flow",
    "description": "Perform multiple Transfer Actions",
    "visible_to": ["urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1"],
    "runnable_by": ["urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1"],
    #"visible_to": ["public"],
    #"runnable_by": ["public"],
    "definition": {
        "StartAt": "TransferAction",
        "States": {
            # Supports zero Transfers
            "CheckInputs": {
                "Type": "Pass",
                "Result": {
                    "loop_finished.=": "`$.action_inputs` == []"
                    # Can't guarantee simpleeval allows list creation,
                    # other option is to compare to always-empty list slice
                    #"loop_finished.=": "`$.action_inputs` == `$.action_inputs`[0:0]"
                },
                "ResultPath": "$",
                "Next": "ChooseLoop"
            },
            "ChooseLoop": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.loop_finished",
                    "BooleanEquals": True,
                    "Next": "FinishLoop"
                }],
                "Default": "TransferAction"
            }, 
            "TransferAction": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/transfer/transfer",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/transfer/transfer",
                "ExceptionOnActionFailure": True,
                "InputPath": "$.action_inputs[0]",
                "ResultPath": "$.UserTransferResult",
                "WaitTime": 86400,
                "Next": "RemoveInput"
            },
            "RemoveInput": {
                "Type": "Pass",
                "Result": {
                    "action_inputs.=": "`$.action_inputs`[1:]"
                },
                "ResultPath": "$",
                "Next": "CheckInputs"
            },
            "FinishLoop": {
                "Type": "Pass",
                "End": True
            }
        }
    }
}

input_schema = {
    "action_inputs": "list of dict - Transfer AP inputs"
}

In [23]:
transfer_flow_deploy_res = flows_client.deploy_flow(
    flow_definition=transfer_loop_flow_def["definition"],
    title=transfer_loop_flow_def["title"],
    description=transfer_loop_flow_def["description"],
    visible_to=transfer_loop_flow_def["visible_to"],
    runnable_by=transfer_loop_flow_def["runnable_by"],
)
transfer_flow_id = transfer_flow_deploy_res["id"]
transfer_flow_scope = transfer_flow_deploy_res["globus_auth_scope"]
transfer_flow_url = "https://flows.globus.org/flows/" + transfer_flow_id
print(transfer_flow_id)
print(transfer_flow_scope)

GlobusAPIError: (400, 'Error', '{"error":{"code":"FLOW_CONFIGURATION_ERROR","detail":"Unable to find requested scopes: [https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/transfer/transfer]"}}\n')

In [13]:
transfer_flow_input = {
    "action_inputs": [{
        # jgaff_laptop to MDF Connect Dev-P
        "destination_endpoint_id": "ca7550ad-55a9-4762-b558-8f2b15049039",
        "label": "flow_test_a",  # optional
        "source_endpoint_id": "ce6d512a-b414-11e7-b0a7-22000a92523b",
        "transfer_items": [{
            "destination_path": "/home/ubuntu/data/deleteme/",
            "source_path": "/home/jgaff/Downloads/connect_demo/",
            "recursive": True
            }]
    }, {
        # MDF Connect Dev-P to NCSA
        "destination_endpoint_id": "82f1b5c6-6e9b-11e5-ba47-22000b92c6ec",
        "label": "flow_test_b",  # optional
        "source_endpoint_id": "ca7550ad-55a9-4762-b558-8f2b15049039",
        "transfer_items": [{
            "destination_path": "/test/deleteme_outcar",
            "source_path": "/home/ubuntu/data/deleteme/AgAl.hP2/OUTCAR",
            "recursive": False
        }]
    }]
}

In [4]:
flow_permissions = ["urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1"]  # jgaff@uchicago.edu
# flow_permissions = ["urn:globus:groups:"]

In [ ]:
mdf_flow_def = {
    "title": "The Materials Data Facility Dataset Processing Flow",
    "description": "Extract, process, and ingest a dataset into MDF Connect.",
    "visible_to": flow_permissions,
    "runnable_by": flow_permissions,
    "definition": {
        "StartAt": "UserTransfer",
        "States": {
            # There must be at least one data location
            "UserTransfer": {
                "Type": "Action",
                "ActionUrl": transfer_flow_url,
                "ActionScope": transfer_flow_scope,
                "ExceptionOnActionFailure": True,
                "RunAs": "Input",
                "Parameters": {
                    "action_inputs.$": "$.globus_data_locations"
                },
                "ResultPath": "$.UserTransferResult",
                "WaitTime": 86400,
                "Next": "Xtraction"
            },
            "Xtraction": {
                "Type": "Action",
                "ActionUrl": "",
                "ActionScope": "",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    pass
                },
                "ResultPath": "$.XtractionResult",
                "WaitTime": 86400,
                "Next": "FeedstockTransfer"
            },
            "FeedstockTransfer": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/transfer/transfer",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/transfer/transfer",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "destination_endpoint_id.$": "$.feedstock_ep",
                    # TODO: Xtract output EP
                    "source_endpoint_id": "",
                    "transfer_items": [{
                        # TODO: Xtract filename
                        "destination_path.=": "`$.feedstock_dir` + ",
                        # TODO: Xtract output path
                        "source_path": "",
                        "recursive": False
                    }]
                },
                "ResultPath": "$.FeedstockTransferResult",
                "WaitTime": 86400,
                "Next": "ChooseCuration"
            },
            "ChooseCuration": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.curation_input",
                    "BooleanEquals": False,
                    "Next": "SearchIngest"
                }],
                "Default": "CurateSubmission"
            },
            "CurateSubmission": {
                # TODO: Update Curation Flow
                "Type": "Action",
                "ActionUrl": curation_flow_url,
                "ActionScope": curation_flow_scope,
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    pass
                },
                "ResultPath": "$.CurationResult",
                "WaitTime": 86400,
                "Next": "ChooseAcceptance"
            },
            "ChooseAcceptance": {
                # TODO: Update Curation Flow
                "Type": "Choice",
                "Choices": [{
                    pass
                }]
                "Default": ""
            },
            "SearchIngest": {
                # TODO: Find docs for this
                pass
            },
            "DataDestTransfer":{
                "Type": "Action",
                "ActionUrl": transfer_flow_url,
                "ActionScope": transfer_flow_scope,
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "action_inputs.$": "$.data_destinations"
                },
                "ResultPath": "$.DataDestResult",
                "WaitTime": 86400,
                "Next": "ChoosePublish"
            },
            "ChoosePublish": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.mdf_publish",
                    "BooleanEquals": True,
                    "Next": "MDFPublish"
                }],
                "Default": "ChooseCitrine"
            },
            "MDFPublish": {
                # TODO: Mint DOI + ?
                "Type": "Action",
                "ActionUrl": "",
                "ActionScope": "",
                "ExceptionOnActionFailure": False,
                "Parameters": {
                    pass
                },
                "ResultPath": "$.MDFPublishResult",
                "WaitTime": 86400,
                "Next": "ChooseCitrine"
            },
            "ChooseCitrine": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.citrine",
                    "BooleanEquals": True,
                    "Next": "CitrinePublish"
                }],
                "Default": "ChooseMRR"
            },
            "CitrinePublish": {
                # FuncX function fo publish to Citrine
                "Type": "Action",
                "ActionUrl": "",
                "ActionScope": "",
                "ExceptionOnActionFailure": False,
                "Parameters": {
                    pass
                },
                "ResultPath": "$.CitrinePublishResult",
                "WaitTime": 86400,
                "Next": "ChooseMRR"
            },
            "ChooseMRR": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.mrr",
                    "BooleanEquals": True,
                    "Next": "MRRPublish"
                }],
                "Default": "SearchUpdate"
            },
            "MRRPublish":{
                # FuncX function to publish to MRR
                "Type": "Action",
                "ActionUrl": "",
                "ActionScope": "",
                "ExceptionOnActionFailure": False,
                "Parameters": {
                    pass
                },
                "ResultPath": "$.MRRPublishResult",
                "WaitTime": 86400,
                "Next": "SearchUpdate"
            },
            "SearchUpdate": {
                # TODO: How to fetch previous dataset entry - Xtract results?
                # TODO: Apply services changes to dataset entry
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/search/ingest",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/search/ingest",
                "ExceptionOnActionFailure": False,
                "Parameters": {
                    # TODO
                    "subject": "",
                    "content": {},
                    "visible_to": [],
                    "search_index": ""
                    
                },
                "ResultPath": "$.SearchUpdateResult",
                "WaitTime": 86400,
                "Next": "FinishSubmission"
            },
            "FinishSubmission": {
                "Type": "Pass",
                "End": True
            }
        }
    }
}

input_schema = {
    pass
}

In [ ]:
flow_deploy_res = flows_client.deploy_flow(
    flow_definition=mdf_flow_def["definition"],
    title=mdf_flow_def["title"],
    description=mdf_flow_def["description"],
    visible_to=mdf_flow_def["visible_to"],
    runnable_by=mdf_flow_def["runnable_by"],
)
flow_deploy_res

In [ ]:
flow_id = flow_deploy_res["id"]
flow_scope = flow_deploy_res["globus_auth_scope"]
print(flow_id)
print(flow_scope)

In [ ]:
flow_input = {
    pass
}

In [ ]:
flow_res = flows_client.run_flow(flow_id, flow_scope, flow_input)
flow_res.data

In [ ]:
status = flows_client.flow_action_status(flow_id, flow_scope, flow_res["action_id"]).data
print(json.dumps(status, indent=4, sort_keys=True))

In [ ]:
flows_client.flow_action_log(flow_id, flow_scope, flow_res["action_id"], limit=100).data